In [1]:
import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd

import acquire as a
import os
import acquire_news_articles as n

**1. Define a function named basic_clean. It should take in a string and apply some basic text cleaning to it:**

Lowercase everything

Normalize unicode characters

Replace anything that is not a letter, number, whitespace or a single quote. 

In [2]:
# 1. lowercase

def basic_clean(df):
    for row in df:
        df[row] = df[row].apply(str.lower)
        df[row] = df[row].map(lambda x: unicodedata.normalize('NFKD', x))
        df[row] = df[row].map(lambda x: x.encode('ascii', 'ignore'))
        df[row] = df[row].map(lambda x: x.decode('utf-8'))
        df[row] = df[row].map(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
    return df
    

In [3]:
original = "Gautam Adani's 3 firms planning to raise $5 billion: Report"

In [4]:
inshorts = a.get_inshorts_df()
len(inshorts)

100

In [5]:
n.get_entertainment_df()

,title,content,category
0,Deepika reacts to political backlash against h...,Deepika Padukone has reacted to the various po...,entertainment
1,"Fought cancer, was told I had 2-3 years left: ...","Dolph Lundgren, who was seen in films like 'Ro...",entertainment
2,"Rashmika trolled for promoting chicken in ad, ...",Rashmika Mandanna has been trolled for a burge...,entertainment
3,Beyoncé wears colour-changing dress during co...,Singer Beyoncé wore a colour-changing dress d...,entertainment
4,"Blackmail, cheap publicity: 'Taarak Mehta...' ...",'Taarak Mehta Ka Ooltah Chashmah' producer Asi...,entertainment
5,'Taarak Mehta...' actress Jennifer accuses pro...,"TV actress Jennifer Mistry Bansiwal, who playe...",entertainment
6,"Complaint filed against Prabhas, Kriti Sanon's...",A complaint has been filed against Prabhas and...,entertainment
7,I don't give a f*** who he has dated: Priyanka...,Priyanka Chopra revealed she never got bothere...,entertainment
8,"Gauahar Khan, Zaid Darbar blessed with a baby boy",Actress Gauahar Khan and her husband Zaid Darb...,entertainment
9,TMKOC's Mandar reacts to Jennifer accusing pro...,Reacting to Jennifer Mistry Bansiwal accusing ...,entertainment


In [6]:
basic_clean = basic_clean(inshorts)
basic_clean

,title,content,category
0,rrs yashasvi jaiswal smashes fastest fifty in ...,rajasthan royals rr opener yashasvi jaiswal on...,sports
1,rr break record for scoring most runs in 1st o...,rr on thursday broke the record for scoring mo...,sports
2,rr to bowl first against kkr trent boult retur...,rajasthan royals rr captain sanju samson won t...,sports
3,i came home crying shaved my head kkrs suyash ...,talking about not being selected after giving ...,sports
4,rajasthan royals record biggest win of ipl 202...,rajasthan royals rr on thursday recorded the b...,sports
...,...,...,...
20,i never made wrong decisions kangana on rascal...,kangana ranaut reacted to an old video shared ...,entertainment
21,sonam is gutsy deepika isnt adventurous design...,while talking about hindi film actresses being...,entertainment
22,want to ask amitabh bachchan where do you want...,actor piyush mishra said that he wants to ask ...,entertainment
23,decision to protect people of wb tmc on bannin...,tmc leader madan mitra on thursday said the ke...,entertainment


In [7]:
article_list = n.get_news_articles()

In [8]:
article_list

[[{'title': "RR's Yashasvi Jaiswal smashes fastest fifty in IPL history",
   'content': 'Rajasthan Royals (RR) opener Yashasvi Jaiswal on Thursday broke the record for slamming the fastest fifty in the history of IPL. The 21-year-old reached his half-century off 13 deliveries against Kolkata Knight Riders (KKR) at Eden Gardens in Kolkata. The previous record was jointly held by KL Rahul and Pat Cummins, who had taken 14 deliveries each.\n\n\n\n\n\n',
   'category': 'sports'},
  {'title': 'RR break record for scoring most runs in 1st over of IPL innings without extras',
   'content': "RR on Thursday broke the record for scoring most runs in the first over of an IPL innings without extras. RR scored 26 runs on the bowling of KKR captain Nitish Rana in their first over with all the runs coming through Yashasvi Jaiswal's bat. The previous record was held by RCB (23 runs against RR in IPL 2019).",
   'category': 'sports'},
  {'title': "RR to bowl first against KKR, Trent Boult returns to RR

**2. Define a function named tokenize. It should take in a string and tokenize all the words in the string.**

In [9]:
def tokenize(df):
    tokenizer = nltk.tokenize.ToktokTokenizer()
    for row in df:
        df[row] = df[row].map(lambda x: tokenizer.tokenize(x, return_str=True))
    return df

In [10]:
tokenized = tokenize(basic_clean)

**3. Define a function named stem. It should accept some text and return the text after applying stemming to all the words.**

In [11]:
def stem(df):
    stemmer = nltk.porter.PorterStemmer()
    for row in df:
        df[row] = df[row].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))
    return df

In [12]:
stem(tokenized)

,title,content,category
0,rr yashasvi jaiswal smash fastest fifti in ipl...,rajasthan royal rr open yashasvi jaiswal on th...,sport
1,rr break record for score most run in 1st over...,rr on thursday broke the record for score most...,sport
2,rr to bowl first against kkr trent boult retur...,rajasthan royal rr captain sanju samson won th...,sport
3,i came home cri shave my head kkr suyash sharm...,talk about not be select after give trial for ...,sport
4,rajasthan royal record biggest win of ipl 2023...,rajasthan royal rr on thursday record the bigg...,sport
...,...,...,...
20,i never made wrong decis kangana on rascal dou...,kangana ranaut react to an old video share by ...,entertain
21,sonam is gutsi deepika isnt adventur design ab...,while talk about hindi film actress be adventu...,entertain
22,want to ask amitabh bachchan where do you want...,actor piyush mishra said that he want to ask a...,entertain
23,decis to protect peopl of wb tmc on ban kerala...,tmc leader madan mitra on thursday said the ke...,entertain


**4. Define a function named lemmatize. It should accept some text and return the text after applying lemmatization to each word.**

In [13]:
def lemmatize(df):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    for row in df:
        df[row] = df[row].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
    return df

In [14]:
lemmatized = lemmatize(tokenized)
lemmatized

,title,content,category
0,rr yashasvi jaiswal smash fastest fifti in ipl...,rajasthan royal rr open yashasvi jaiswal on th...,sport
1,rr break record for score most run in 1st over...,rr on thursday broke the record for score most...,sport
2,rr to bowl first against kkr trent boult retur...,rajasthan royal rr captain sanju samson won th...,sport
3,i came home cri shave my head kkr suyash sharm...,talk about not be select after give trial for ...,sport
4,rajasthan royal record biggest win of ipl 2023...,rajasthan royal rr on thursday record the bigg...,sport
...,...,...,...
20,i never made wrong decis kangana on rascal dou...,kangana ranaut react to an old video share by ...,entertain
21,sonam is gutsi deepika isnt adventur design ab...,while talk about hindi film actress be adventu...,entertain
22,want to ask amitabh bachchan where do you want...,actor piyush mishra said that he want to ask a...,entertain
23,decis to protect peopl of wb tmc on ban kerala...,tmc leader madan mitra on thursday said the ke...,entertain


**5. Define a function named remove_stopwords. It should accept some text and return the text after removing all the stopwords.**

In [15]:
# to compare elements in different things, we can use 'set'

set1 = set([1,2,3,4,5,6])
set2 = set([5,5,6,7,8])

In [16]:
set2 - set1

{7, 8}

In [17]:
set1 - set2

{1, 2, 3, 4}

In [18]:
# finding the unique elemets

set1.union(set2)

{1, 2, 3, 4, 5, 6, 7, 8}

In [19]:
def remove_stopwords(df, extra_words = [], keep_words = []):
    nltk.stopwords = nltk.corpus.stopwords
    stopwords_custom = set(stopwords.words('english')) - set(keep_words)
    stopwords_custom = list(stopwords_custom.union(extra_words))
    for row in df:
        df[row] = df[row].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords_custom]))
    return df

In [20]:
remove_stopwords(lemmatized, extra_words=['trauma', 'smash'])

,title,content,category
0,rr yashasvi jaiswal fastest fifti ipl histori,rajasthan royal rr open yashasvi jaiswal thurs...,sport
1,rr break record score run 1st ipl inning witho...,rr thursday broke record score run first ipl i...,sport
2,rr bowl first kkr trent boult return rr play xi,rajasthan royal rr captain sanju samson toss e...,sport
3,came home cri shave head kkr suyash sharma u19...,talk select give trial under19 last year kkr s...,sport
4,rajasthan royal record biggest win ipl 2023 te...,rajasthan royal rr thursday record biggest win...,sport
...,...,...,...
20,never made wrong decis kangana rascal doubl dh...,kangana ranaut react old video share fan page ...,entertain
21,sonam gutsi deepika isnt adventur design abusa...,talk hindi film actress adventur fashion choic...,entertain
22,want ask amitabh bachchan want stop piyush,actor piyush mishra said want ask amitabh bach...,entertain
23,decis protect peopl wb tmc ban kerala stori,tmc leader madan mitra thursday said kerala st...,entertain


**6. Use your data from the acquire to produce a dataframe of the news articles. Name the dataframe news_df.**

In [21]:
news_df = a.get_inshorts_df()

In [22]:
news_df

,title,content,category
0,RR's Yashasvi Jaiswal smashes fastest fifty in...,Rajasthan Royals (RR) opener Yashasvi Jaiswal ...,sports
1,RR break record for scoring most runs in 1st o...,RR on Thursday broke the record for scoring mo...,sports
2,"RR to bowl first against KKR, Trent Boult retu...",Rajasthan Royals (RR) captain Sanju Samson won...,sports
3,"I came home crying, shaved my head: KKR's Suya...",Talking about not being selected after giving ...,sports
4,Rajasthan Royals record biggest win of IPL 202...,Rajasthan Royals (RR) on Thursday recorded the...,sports
...,...,...,...
20,I never made wrong decisions: Kangana on Rasca...,Kangana Ranaut reacted to an old video shared ...,entertainment
21,"Sonam is gutsy, Deepika isn't adventurous: Des...",While talking about Hindi film actresses being...,entertainment
22,Want to ask Amitabh Bachchan 'Where do you wan...,Actor Piyush Mishra said that he wants to ask ...,entertainment
23,Decision to protect people of WB: TMC on banni...,TMC leader Madan Mitra on Thursday said 'The K...,entertainment


**7. Make another dataframe for the Codeup blog posts. Name the dataframe codeup_df.**

In [23]:
codeup_df = a.get_blog_df()

In [24]:
codeup_df

,title,content
0,Women in tech: Panelist Spotlight – Magdalena ...,\nCodeup is hosting a Women in Tech Panel in h...
1,Women in tech: Panelist Spotlight – Rachel Rob...,\nCodeup is hosting a Women in Tech Panel in h...
2,Women in Tech: Panelist Spotlight – Sarah Mellor,\nCodeup is hosting a Women in Tech Panel in ...
3,Women in Tech: Panelist Spotlight – Madeleine ...,\nCodeup is hosting a Women in Tech Panel in h...
4,Black Excellence in Tech: Panelist Spotlight –...,\n\nCodeup is hosting a Black Excellence in Te...
5,Black excellence in tech: Panelist Spotlight –...,\nCodeup is hosting our second Black Excellenc...


**8. For each dataframe, produce the following columns:**

title to hold the title

original to hold the original article/post content

clean to hold the normalized and tokenized original with the stopwords removed.

stemmed to hold the stemmed version of the cleaned data.

lemmatized to hold the lemmatized version of the cleaned data.

In [34]:
def transform_data(df):
    df = df.rename(columns={'content': 'original'})
    df['clean'] = df['original'].apply(basic_clean).apply(tokenize).apply(remove_stopwords)
    return df

In [26]:

def basic_clean(df):
    for row in df:
        df[row] = df[row].apply(str.lower)
        df[row] = df[row].map(lambda x: unicodedata.normalize('NFKD', x))
        df[row] = df[row].map(lambda x: x.encode('ascii', 'ignore'))
        df[row] = df[row].map(lambda x: x.decode('utf-8'))
        df[row] = df[row].map(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
    return df

In [27]:
basic_clean(codeup_df)

,title,content
0,women in tech panelist spotlight magdalena rahn,\ncodeup is hosting a women in tech panel in h...
1,women in tech panelist spotlight rachel robbi...,\ncodeup is hosting a women in tech panel in h...
2,women in tech panelist spotlight sarah mellor,\ncodeup is hosting a women in tech panel in ...
3,women in tech panelist spotlight madeleine ca...,\ncodeup is hosting a women in tech panel in h...
4,black excellence in tech panelist spotlight w...,\n\ncodeup is hosting a black excellence in te...
5,black excellence in tech panelist spotlight s...,\ncodeup is hosting our second black excellenc...


In [35]:
transform_data(codeup_df)

TypeError: string indices must be integers